# DESAFIO FINAL!

In [144]:
# Importando as Bibliotecas

import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm

In [145]:
# Lendo os arquivos
csv_market = pd.read_csv('estaticos_market.csv')
csv_portfolio1 = pd.read_csv('portfolio1.csv')
csv_portfolio2 = pd.read_csv('portfolio2.csv')
csv_portfolio3 = pd.read_csv('portfolio3.csv')

# Criando DataFrame para cada um dos arquivos
df_market = pd.DataFrame(csv_market)
df_portfolio1 = pd.DataFrame(csv_portfolio1)
df_portfolio2 = pd.DataFrame(csv_portfolio2)
df_portfolio3 = pd.DataFrame(csv_portfolio3)

In [146]:
df_market.shape

(462298, 182)

In [147]:
df_portfolio1.shape

(555, 182)

In [148]:
df_portfolio2.shape

(566, 2)

### ENTENDENDO OS DADOS

### Tratando os Dados: Modo Geral
#### > Excluindo as colunas com mais de 70% dados faltantes
#### > Pandas Profiling: 
##### - Verificando as colunas que são correlacionadas entre si e excluindo uma das opções;
##### - Excluindo colunas que tem mais de 90% de zeros.

In [149]:
# Criando DataFrame auxiliar: verificar a consistência dos dados
df_market_tamanho = df_market.shape[0]
df_market_consistencia = pd.DataFrame({'COLUNAS': df_market.columns, 'TIPOS': df_market.dtypes, 'FALTANTES': df_market.isna().sum(), 'FALTANTES_PERCENTUAL': round(df_market.isna().sum()/df_market_tamanho,2)})

In [150]:
df_market.shape

(462298, 182)

In [151]:
df_portfolio3.shape

(265, 2)

In [152]:
# Acrescentando uma coluna no DataFrame para informar se algum atrituto tem mais de 70% de dados faltantes
df_market_consistencia['TemMaisDe70%Faltantes'] = ['sim' if x > 0.7 else 'nao' for x in df_market_consistencia['FALTANTES_PERCENTUAL']]

In [153]:
df_market_consistencia.head()

,COLUNAS,TIPOS,FALTANTES,FALTANTES_PERCENTUAL,TemMaisDe70%Faltantes
Unnamed: 0,Unnamed: 0,int64,0,0.0,nao
id,id,object,0,0.0,nao
fl_matriz,fl_matriz,bool,0,0.0,nao
de_natureza_juridica,de_natureza_juridica,object,0,0.0,nao
sg_uf,sg_uf,object,0,0.0,nao


In [154]:
# Excluindo as colunas que tem mais de 70% de dados faltantes
manter_semFaltantes = list(df_market_consistencia[df_market_consistencia['TemMaisDe70%Faltantes'] == 'nao']['COLUNAS'])
df_market = df_market[manter_semFaltantes]

In [155]:
# Verificando quantas colunas excluimos
# Descartamos 129 variáveis!
df_market.shape

(462298, 53)

In [156]:
df_market.dtypes.value_counts()

object     26
float64    16
bool        9
int64       2
dtype: int64

In [157]:
#profile = ProfileReport(df_market, title="Pandas Profiling Report", html={'style':{'full_width':True}})

In [158]:
#profile.to_notebook_iframe()

In [159]:
#profile.to_file("report_sem_70_faltantes.html")

In [160]:
# Selecionando e excluindo colunas com as seguintes caracteristicas:
# constantes, com mais de 90% de valor 'zero' e mais de 50% de dados faltantes
lista_excluir_consEzerosE50faltantes = ['fl_epp', 'vl_total_veiculos_pesados_grupo', 'vl_total_veiculos_leves_grupo', 'qt_socios_pj', 'qt_filiais', 'qt_socios_feminino', 'qt_socios_masculino']
df_market.drop(columns = lista_excluir_consEzerosE50faltantes, inplace=True)

In [161]:
# Fomos de 53 para 46 atributos
df_market.shape

(462298, 46)

In [162]:
# VERIFICAR: 
# idade_maxima_socios  <---> idade_media_socios (> idade_minima_socios > ?)
# idade_minima_socios   ---> idade_media_socios
# PENSAR: Qual excluir?

In [163]:
df_market[['idade_media_socios', 'idade_maxima_socios', 'idade_minima_socios']].head()

,idade_media_socios,idade_maxima_socios,idade_minima_socios
0,44.0,47.0,41.0
1,27.0,27.0,27.0
2,32.0,32.0,32.0
3,36.0,36.0,36.0
4,NaN,NaN,NaN


In [164]:
print('UNICOS: media',df_market['idade_media_socios'].unique())
print('UNICOS: min',df_market['idade_minima_socios'].unique())
print('UNICOS: max',df_market['idade_maxima_socios'].unique())

UNICOS: media [44.         27.         32.         ... 60.71428571 74.83333333
 43.55555556]
UNICOS: min [ 41.  27.  32.  36.  nan  63.  40.  52.  62.  47.  34.  55.  64.  24.
  45.  26.  23.  37.  44.  25.  68.  66.  28.  30.  33.  65.  35.  46.
  48.  31.  43.  84.  42.  53.  29.  49.  54.  59.  57.  38.  22.  75.
  19.  51.  58.  56.  39.  21.  60.  83.  85.  50.  86.  18.  69.  79.
  74.  61.  20.  70.  87.  72.  78.  77.  11.  76.  71.  17.  73.  67.
  15. 104.  81.  80.  92.   8.  88.  12.  96.  94.  82.  90.  93.  16.
  10. 110.  13.  91.  89.  14.   2.   7. 106.  97.  99.   5. 100.  98.
   4.   6.   9. 121. 108.  95. 102. 103. 117. 127. 101.   3. 105. 111.
  -2. 109. 112.]
UNICOS: max [ 47.  27.  32.  36.  nan  63.  40.  52.  62.  34.  55.  64.  24.  45.
  26.  48.  23.  37.  73.  75.  25.  41.  68.  66.  28.  30.  33.  65.
  35.  46.  44.  61.  31.  43.  84.  54.  67.  42.  57.  29.  49.  60.
  59.  38.  19.  53.  56.  71.  39.  21.  51.  72.  83.  58.  50.  85.
  74.  22.  86

In [165]:
# VERIFICAR: CORRELAÇÃO
# idade_maxima_socios  <---> idade_media_socios (> idade_minima_socios > ?)
# idade_minima_socios   ---> idade_media_socios
# PENSAR: Qual excluir?
# DECISÃO: Excluir (idade_maxima_socios, idade_minima_socios)

lista_excluir_corr_idade = ['idade_maxima_socios', 'idade_minima_socios']
df_market.drop(columns = lista_excluir_corr_idade, inplace=True)

In [166]:
# Fomos de 46 para 44 colunas!
df_market.shape

(462298, 44)

In [167]:
# VERIFICAR: CORRELAÇÃO
# sg_uf_matriz <---> sg_uf (mais 2 correlações)
# DECISÃO: excluir "sg_uf_matriz"
lista_excluir_corr_UF = ['sg_uf_matriz']
df_market.drop(columns = lista_excluir_corr_UF, inplace=True)

In [168]:
# Fomos de 44 para 43!
df_market.shape

(462298, 43)

In [169]:
df_correlacao = df_market.corr(method='pearson')

In [170]:
# VERIFICAR: CORRELAÇÃO
# Resultado: altamente correlacionados: qt_socios, qt_socios_st_regular, qt_socios_pf
# Como: (method='pearson')
# DECISÃO: excluir: qt_socios_st_regular, qt_socios_pf
df_correlacao = df_market.corr(method='pearson')
df_correlacao['qt_socios_st_regular']


Unnamed: 0                               0.002565
fl_matriz                               -0.230071
idade_empresa_anos                       0.105316
fl_me                                   -0.002700
fl_sa                                    0.342961
fl_mei                                  -0.148889
fl_ltda                                  0.031683
fl_st_especial                           0.000919
fl_email                                -0.023276
fl_telefone                             -0.042685
nu_meses_rescencia                       0.017897
empsetorcensitariofaixarendapopulacao    0.044481
qt_socios                                0.986359
qt_socios_pf                             0.988362
idade_media_socios                       0.066835
qt_socios_st_regular                     1.000000
vl_faturamento_estimado_aux              0.033405
vl_faturamento_estimado_grupo_aux        0.807327
Name: qt_socios_st_regular, dtype: float64

In [171]:
# VERIFICAR: CORRELAÇÃO
# Resultado: altamente correlacionados: qt_socios, qt_socios_st_regular, qt_socios_pf
# Como:
# DECISÃO: excluir: qt_socios_st_regular, qt_socios_pf
# Executando a decisão:
lista_excluir_corr_socios = ['qt_socios_st_regular', 'qt_socios_pf']
df_market.drop(columns = lista_excluir_corr_socios, inplace=True)

In [172]:
# Fomos de 43 colunas para 41
df_market.shape

(462298, 41)

In [173]:
df_market.drop(columns = 'dt_situacao', inplace = True)
df_market.shape

(462298, 40)

In [174]:
df_market.isna().sum()

Unnamed: 0                                    0
id                                            0
fl_matriz                                     0
de_natureza_juridica                          0
sg_uf                                         0
natureza_juridica_macro                       0
de_ramo                                       0
setor                                      1927
idade_empresa_anos                            0
idade_emp_cat                                 0
fl_me                                         0
fl_sa                                         0
fl_mei                                        0
fl_ltda                                       0
fl_st_especial                                0
fl_email                                      0
fl_telefone                                   0
fl_rm                                         0
nm_divisao                                 1927
nm_segmento                                1927
fl_spa                                  

In [175]:
# Anotações do Pandas Profiling : o que fazer considerando os dados faltantes obervados acima!
# OBS: o que esta entre parenteses é a quantidade de dados faltantes!
# setor (1927) <---> de_ramo (0) : excluir a coluna "setor"
# nm_meso_regiao (58698) ---> sg_uf (0) : excluir coluna "nm_meso_regiao"
# nm_micro_regiao(58698) ---> sg_uf (0) : excluir coluna "nm_micro_regiao"
# fl_rm (0) ---> nm_micro_regiao : ok
# natureza_juridica_macro(0) <---> de_natureza_juridica(0) : ok (por enquanto nada)
lista_excluir_OBJ = ['setor', 'nm_meso_regiao', 'nm_micro_regiao' ]

In [176]:
# Excluindo o que foi definido acima!
df_market.drop(columns = lista_excluir_OBJ, inplace = True)

In [177]:
df_market.shape

(462298, 37)

In [178]:
# Analisando os dados unicos das colunas "que sobraram"
# Resultado: Excluir as seguintes colunas:
lista_excluir_unicosEmuitos = ['fl_me', 'fl_st_especial', 'qt_socios', 'empsetorcensitariofaixarendapopulacao', 'idade_media_socios', 'de_faixa_faturamento_estimado', 'vl_faturamento_estimado_grupo_aux','fl_spa', 'fl_antt', 'fl_veiculo', 'fl_simples_irregular', 'fl_passivel_iss', 'fl_optante_simei']
df_market.drop(columns = lista_excluir_unicosEmuitos, inplace = True)


In [179]:
# Verificando a quantidade de colunas "que sobrou"
# Resultado: diminuir bastante! de 37 para 21
df_market.shape

(462298, 24)

In [180]:
# Verificando a quantidade de dados faltantes de cada columa
# Resultado: temos algumas colunas com dados faltantes! Precisamos imputar dados.
df_market.isna().sum()

Unnamed: 0                                 0
id                                         0
fl_matriz                                  0
de_natureza_juridica                       0
sg_uf                                      0
natureza_juridica_macro                    0
de_ramo                                    0
idade_empresa_anos                         0
idade_emp_cat                              0
fl_sa                                      0
fl_mei                                     0
fl_ltda                                    0
fl_email                                   0
fl_telefone                                0
fl_rm                                      0
nm_divisao                              1927
nm_segmento                             1927
fl_optante_simples                     82713
de_saude_tributaria                    14851
de_saude_rescencia                     14851
nu_meses_rescencia                     45276
de_nivel_atividade                     11168
de_faixa_f

### SEPARANDO AS COLUNAS POR TIPOS!

In [181]:
# Verificando os tipos dos dados presente neste DataFrame
df_market.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('bool'), dtype('float64')],
      dtype=object)

In [182]:
df_market_tipos_aux = pd.DataFrame({'colunas': df_market.columns, 'tipos':df_market.dtypes})
df_market_tipos_aux.head()

,colunas,tipos
Unnamed: 0,Unnamed: 0,int64
id,id,object
fl_matriz,fl_matriz,bool
de_natureza_juridica,de_natureza_juridica,object
sg_uf,sg_uf,object


In [183]:
# Separando os tipos em listas!
lista_dadosNumericos = list(df_market_tipos_aux[(df_market_tipos_aux['tipos'] == 'int64') | (df_market_tipos_aux['tipos'] == 'float64')]['colunas'])
lista_dadosBool = list(df_market_tipos_aux[(df_market_tipos_aux['tipos'] == 'bool')]['colunas'])
lista_dadosObject = list(df_market_tipos_aux[(df_market_tipos_aux['tipos'] == 'object')]['colunas'])

In [184]:
# Criando DataFrames de acordo com as listas acima!
df_market_dadosNumericos = df_market[lista_dadosNumericos]
df_market_dadosBool = df_market[lista_dadosBool]
df_market_dadosObject = df_market[lista_dadosObject]

# (df_market_todoNumerico)! É o DataFrame todo numérico para conseguirmos trabalhar!

In [185]:
df_market_dadosObject.columns

Index(['id', 'de_natureza_juridica', 'sg_uf', 'natureza_juridica_macro',
       'de_ramo', 'idade_emp_cat', 'fl_rm', 'nm_divisao', 'nm_segmento',
       'fl_optante_simples', 'de_saude_tributaria', 'de_saude_rescencia',
       'de_nivel_atividade', 'de_faixa_faturamento_estimado_grupo'],
      dtype='object')

### Trabalhando com DADOS "bool"

In [186]:
# Transformando os dados "bool" em "int"
df_market_dadosBool_transformado = df_market_dadosBool.astype(int)
df_market_dadosBool_transformado.head()

,fl_matriz,fl_sa,fl_mei,fl_ltda,fl_email,fl_telefone
0,1,0,0,0,1,1
1,1,0,1,0,1,1
2,1,0,1,0,0,1
3,1,0,0,0,1,1
4,1,0,0,0,1,1


In [187]:
# Adicionando (df_market_dadosBool_transformado) ao (df_market_todoNumerico)
df_market_todoNumerico = df_market_dadosBool_transformado

In [188]:
# Verificando se o (df_market_dadosBool_transformado) foi adicionado corretamente em (df_market_todoNumerico)
df_market_todoNumerico.shape

(462298, 6)

In [189]:
# Verificando a quantidade de dados faltantes no (df_market_todoNumerico). Continua zerado. ÓTIMO!
df_market_todoNumerico.isna().sum()

fl_matriz      0
fl_sa          0
fl_mei         0
fl_ltda        0
fl_email       0
fl_telefone    0
dtype: int64

### Trabalhando com DADOS NUMÉRICOS : "int64"e "float64"

In [190]:
df_market_todoNumerico.columns

Index(['fl_matriz', 'fl_sa', 'fl_mei', 'fl_ltda', 'fl_email', 'fl_telefone'], dtype='object')

In [191]:
df_market_dadosNumericos.isna().sum()

Unnamed: 0                         0
idade_empresa_anos                 0
nu_meses_rescencia             45276
vl_faturamento_estimado_aux    27513
dtype: int64

In [192]:
# Substituindo todos os dados numéricos faltantes por "999999999"
# Dessa forma vamos conseguir usar algum algoritmo de classificacao e mesmo assim vamos saber que é NaN
df_market_dadosNumericos = df_market_dadosNumericos.fillna(999999999)

In [193]:
# Testando para ver se funcionou.
# Verificando a quantidade de dados faltantes nas colunas numéricas. Total = 0, uhuuuL
df_market_dadosNumericos.isna().sum()

Unnamed: 0                     0
idade_empresa_anos             0
nu_meses_rescencia             0
vl_faturamento_estimado_aux    0
dtype: int64

In [194]:
# Adicionando (df_market_dadosNumericos) ao (df_market_todoNumerico)
df_market_todoNumerico = df_market_todoNumerico.join(df_market_dadosNumericos)

In [195]:
# Verificando o número de linhas e colunas no DataFrame (df_market_todoNumerico)
df_market_todoNumerico.shape

(462298, 10)

### >>> IMPUTANDO VALORES: 'int64' e 'float64' <<<

In [196]:
from sklearn.linear_model import LinearRegression

In [197]:
reg = LinearRegression()

### Coluna: nu_meses_rescencia

In [198]:
df_market_TN_treino = df_market_todoNumerico[df_market_todoNumerico['nu_meses_rescencia'] != 999999999]
df_market_TN_teste = df_market_todoNumerico[df_market_todoNumerico['nu_meses_rescencia'] == 999999999]

In [199]:
# Definindo o TREINO
Y_TREINO_df_nu_meses_rescencia = df_market_TN_treino['nu_meses_rescencia']
X_TREINO_df_nu_meses_rescencia = df_market_TN_treino.drop(columns = 'nu_meses_rescencia')

# Definindo TESTE
X_TESTE_df_nu_meses_rescencia = df_market_TN_teste.drop(columns = 'nu_meses_rescencia')

In [200]:
reg.fit(X_TREINO_df_nu_meses_rescencia, Y_TREINO_df_nu_meses_rescencia)
Y_PREDITO_nu_meses_rescencia = reg.predict(X_TESTE_df_nu_meses_rescencia)

In [201]:
# Valores de "nu_meses_rescencia" antes da substituição
df_market_todoNumerico[df_market_todoNumerico['nu_meses_rescencia'] == 999999999].head()

,fl_matriz,fl_sa,fl_mei,fl_ltda,fl_email,fl_telefone,Unnamed: 0,idade_empresa_anos,nu_meses_rescencia,vl_faturamento_estimado_aux
7,1,0,1,0,1,1,7,0.052055,999999999.0,999999999.0
30,1,0,1,0,1,1,30,0.742466,999999999.0,210000.0
32,1,0,1,0,0,1,32,0.161644,999999999.0,999999999.0
36,1,0,0,0,1,1,36,0.375342,999999999.0,999999999.0
39,0,0,0,0,1,1,39,0.082192,999999999.0,0.0


In [202]:
# Substituindo '999999999' pelos valores calculado' no dataFrame (df_market_todoNumerico)
df_market_todoNumerico['nu_meses_rescencia'].replace({999999999: Y_PREDITO_nu_meses_rescencia}, inplace = True)

In [203]:
# Salvando o resultado da imputacão de valores em: (df_market_dadosNumericos_Transformado)
df_market_dadosNumericos_Transformado = pd.DataFrame(df_market_dadosNumericos)

In [204]:
df_market_dadosNumericos_Transformado.drop(columns= 'Unnamed: 0')

,idade_empresa_anos,nu_meses_rescencia,vl_faturamento_estimado_aux
0,14.457534,23.0,3132172.8
1,1.463014,9.0,210000.0
2,7.093151,26.0,50000.0
3,6.512329,26.0,210000.0
4,3.200000,27.0,210000.0
...,...,...,...
462293,3.254795,22.0,50000.0
462294,8.953425,23.0,210000.0
462295,20.745205,23.0,210000.0
462296,1.909589,20.0,432734.4


In [205]:
# Substituindo '999999999' pelos valores calculado' no DataFrame (df_market_dadosNumericos)
df_market_dadosNumericos['nu_meses_rescencia'].replace({999999999: Y_PREDITO_nu_meses_rescencia}, inplace = True)

In [206]:
# Substituindo '999999999' pelos valores calculado' no DataFrame (df_market_dadosNumericos)
df_market_dadosNumericos_Transformado['nu_meses_rescencia'].replace({999999999: Y_PREDITO_nu_meses_rescencia}, inplace = True)

In [207]:
# Conferindo se a substituição ocorreu!
df_market_todoNumerico['nu_meses_rescencia'][36]

24.85954582565315

In [208]:
# Verificando se tem alum valor '999999999'(que no caso é NaN) na coluna "nu_meses_rescencia" no DataFrame(df_market_todoNumerico)
(df_market_todoNumerico['nu_meses_rescencia']== 999999999).unique()

array([False])

### Coluna: vl_faturamento_estimado_aux

In [209]:
df_market_TN_treino = df_market_todoNumerico[df_market_todoNumerico['vl_faturamento_estimado_aux'] != 999999999]
df_market_TN_teste = df_market_todoNumerico[df_market_todoNumerico['vl_faturamento_estimado_aux'] == 999999999]

In [210]:
# Definindo o TREINO
Y_TREINO_df_vl_faturamento_estimado_aux = df_market_TN_treino['vl_faturamento_estimado_aux']
X_TREINO_df_vl_faturamento_estimado_aux = df_market_TN_treino.drop(columns = 'vl_faturamento_estimado_aux')

# Definindo TESTE
X_TESTE_df_vl_faturamento_estimado_aux = df_market_TN_teste.drop(columns = 'vl_faturamento_estimado_aux')

In [211]:
reg.fit(X_TREINO_df_vl_faturamento_estimado_aux, Y_TREINO_df_vl_faturamento_estimado_aux)
Y_PREDITO_vl_faturamento_estimado_aux = reg.predict(X_TESTE_df_vl_faturamento_estimado_aux)

In [212]:
Y_PREDITO_vl_faturamento_estimado_aux = Y_PREDITO_vl_faturamento_estimado_aux * (-1)
Y_PREDITO_vl_faturamento_estimado_aux 

array([ 69450.64326316, 728844.04336684,  62596.69643725, ...,
       740756.23828632,  38425.85289452,  48381.03323677])

In [213]:
# Valores de "nu_meses_rescencia" antes da substituição
df_market_todoNumerico['vl_faturamento_estimado_aux']

0         3132172.8
1          210000.0
2           50000.0
3          210000.0
4          210000.0
            ...    
462293      50000.0
462294     210000.0
462295     210000.0
462296     432734.4
462297     210000.0
Name: vl_faturamento_estimado_aux, Length: 462298, dtype: float64

In [214]:
# Substituindo '999999999' pelos valores calculado'
df_market_todoNumerico['vl_faturamento_estimado_aux'].replace({999999999: Y_PREDITO_vl_faturamento_estimado_aux}, inplace = True)

In [215]:
# Substituindo '999999999' pelos valores calculado' no DataFrame (df_market_dadosNumericos_Transformado)
df_market_dadosNumericos_Transformado['vl_faturamento_estimado_aux'].replace({999999999: Y_PREDITO_vl_faturamento_estimado_aux}, inplace = True)

In [216]:
df_market_dadosNumericos_Transformado.drop(columns = 'Unnamed: 0', inplace = True)

In [217]:
# Conferindo se a substituição ocorreu!
df_market_todoNumerico['vl_faturamento_estimado_aux'][32]

728844.0433668448

In [218]:
# Verificando se tem alum valor '999999999'(que no caso é NaN) na coluna "nu_meses_rescencia" no DataFrame(df_market_todoNumerico)
(df_market_todoNumerico['nu_meses_rescencia']== 999999999).unique()

array([False])

In [219]:
df_market_todoNumerico

,fl_matriz,fl_sa,fl_mei,fl_ltda,fl_email,fl_telefone,Unnamed: 0,idade_empresa_anos,nu_meses_rescencia,vl_faturamento_estimado_aux
0,1,0,0,0,1,1,0,14.457534,23.000000,3132172.8
1,1,0,1,0,1,1,1,1.463014,9.000000,210000.0
2,1,0,1,0,0,1,2,7.093151,26.000000,50000.0
3,1,0,0,0,1,1,3,6.512329,26.000000,210000.0
4,1,0,0,0,1,1,4,3.200000,27.000000,210000.0
...,...,...,...,...,...,...,...,...,...,...
462293,1,0,1,0,1,1,462293,3.254795,22.000000,50000.0
462294,1,0,0,0,0,0,462294,8.953425,23.000000,210000.0
462295,1,0,0,0,0,0,462295,20.745205,23.000000,210000.0
462296,1,0,0,0,1,0,462296,1.909589,20.000000,432734.4


In [220]:
df_market_todoNumerico.drop (columns = 'Unnamed: 0', inplace = True)

In [221]:
df_market_todoNumerico.columns

Index(['fl_matriz', 'fl_sa', 'fl_mei', 'fl_ltda', 'fl_email', 'fl_telefone',
       'idade_empresa_anos', 'nu_meses_rescencia',
       'vl_faturamento_estimado_aux'],
      dtype='object')

### >>> IMPUTANDO VALORES: 'object' <<<

In [222]:
# Lembrando que:
# Nosso DataFrame resultado (até o momento) é: df_market_todoNumerico
# O DataFrame (df_market_dadosObject) é o DataFrame com todos os dados do tipo "object"
# A ideia é usar um algoritmo de Classificação (ex: Árvore de Decisão) para substituir os dados faltantes
# Estamos assumindo que os dados faltantes serão representados por: "souNulo"

In [223]:
df_market_todoNumerico.columns

Index(['fl_matriz', 'fl_sa', 'fl_mei', 'fl_ltda', 'fl_email', 'fl_telefone',
       'idade_empresa_anos', 'nu_meses_rescencia',
       'vl_faturamento_estimado_aux'],
      dtype='object')

In [224]:
df_market_dadosObject.isna().sum()

id                                         0
de_natureza_juridica                       0
sg_uf                                      0
natureza_juridica_macro                    0
de_ramo                                    0
idade_emp_cat                              0
fl_rm                                      0
nm_divisao                              1927
nm_segmento                             1927
fl_optante_simples                     82713
de_saude_tributaria                    14851
de_saude_rescencia                     14851
de_nivel_atividade                     11168
de_faixa_faturamento_estimado_grupo    27513
dtype: int64

In [225]:
# Verificando a quantidade de dados faltantes no DataFrame (df_market_dadosObject).
# Conseguimos verificar que as seguintes colunas não tem nenhum dado faltante:
# listaBOOL_semDadosFaltantes = [id, de_natureza_juridica, sg_uf, natureza_juridica_macro, de_ramo, idade_emp_cat, fl_rm]
lista_dadosObject_COM_dadosFaltantes = ['nm_divisao', 'nm_segmento', 'fl_optante_simples', 'de_saude_tributaria','de_saude_rescencia', 'de_nivel_atividade', 'de_faixa_faturamento_estimado_grupo']
lista_dadosObject_SEM_dadosFaltantes = ['de_natureza_juridica', 'sg_uf', 'natureza_juridica_macro', 'de_ramo', 'idade_emp_cat', 'fl_rm']

In [226]:
# Vamos isolar "id"
# Ou seja, salvar em uma variável e excluir do DataFrame (df_market_dadosObject)
# Porque esta coluna não vai fazer parte de nenhum aprendizado, mas precisamos os valores dos "id"s das empresas.
armazenando_ID = df_market_dadosObject['id']

In [227]:
lista_dadosObject_aux = list(df_market_dadosObject.columns)
lista_dadosObject_aux.remove('id')
lista_dadosObject_aux

['de_natureza_juridica',
 'sg_uf',
 'natureza_juridica_macro',
 'de_ramo',
 'idade_emp_cat',
 'fl_rm',
 'nm_divisao',
 'nm_segmento',
 'fl_optante_simples',
 'de_saude_tributaria',
 'de_saude_rescencia',
 'de_nivel_atividade',
 'de_faixa_faturamento_estimado_grupo']

In [228]:
# Substituindo todos os dados numéricos faltantes por 'souNulo'
# Dessa forma vamos conseguir usar algum algoritmo de classificacao e mesmo assim vamos saber que é NaN
df_market_dadosObject = df_market_dadosObject.fillna('souNulo')

In [229]:
df_market_dadosObject.isna().sum()

id                                     0
de_natureza_juridica                   0
sg_uf                                  0
natureza_juridica_macro                0
de_ramo                                0
idade_emp_cat                          0
fl_rm                                  0
nm_divisao                             0
nm_segmento                            0
fl_optante_simples                     0
de_saude_tributaria                    0
de_saude_rescencia                     0
de_nivel_atividade                     0
de_faixa_faturamento_estimado_grupo    0
dtype: int64

In [230]:
df_parcial_dadosObject_SEM_dadosFaltantes = pd.DataFrame(df_market_dadosObject['id'])

In [231]:
lista_dadosObject_SEM_dadosFaltantes

['de_natureza_juridica',
 'sg_uf',
 'natureza_juridica_macro',
 'de_ramo',
 'idade_emp_cat',
 'fl_rm']

In [232]:
# Transformando as colunas em dados numéricos e salvando num novo DataFrame
# SOMENTE AS COLUNAS DO TIPO "BOOL" SEM DADOS FALTANTES!
for x in lista_dadosObject_SEM_dadosFaltantes:
    df_market_aux = pd.get_dummies(df_market_dadosObject[x], prefix = x)
    df_parcial_dadosObject_SEM_dadosFaltantes = df_parcial_dadosObject_SEM_dadosFaltantes.join(df_market_aux)

In [233]:
# Verificando o foermato do DataFrame que tivemos como resultado!
df_parcial_dadosObject_SEM_dadosFaltantes.shape

(462298, 122)

In [234]:
df_parcial_dadosObject_SEM_dadosFaltantes.head()

,id,de_natureza_juridica_ASSOCIACAO PRIVADA,de_natureza_juridica_AUTARQUIA ESTADUAL OU DO DISTRITO FEDERAL,de_natureza_juridica_AUTARQUIA FEDERAL,de_natureza_juridica_AUTARQUIA MUNICIPAL,de_natureza_juridica_CANDIDATO A CARGO POLITICO ELETIVO,de_natureza_juridica_CLUBE FUNDO DE INVESTIMENTO,de_natureza_juridica_COMUNIDADE INDIGENA,de_natureza_juridica_CONDOMINIO EDILICIO,de_natureza_juridica_CONSORCIO DE EMPREGADORES,...,de_ramo_TEXTEIS,"de_ramo_TRANSPORTE, ARMAZENAGEM E CORREIO",idade_emp_cat_1 a 5,idade_emp_cat_10 a 15,idade_emp_cat_15 a 20,idade_emp_cat_5 a 10,idade_emp_cat_<= 1,idade_emp_cat_> 20,fl_rm_NAO,fl_rm_SIM
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [235]:
df_parcial_dadosObject_SEM_dadosFaltantes.shape

(462298, 122)

In [236]:
# Precisamos excluir o "id" do DataFrame que vamos utilizar para realizar a classificacao
# Porque nao queremos que "id" interfira.
df_parcial_dadosObject_SEM_dadosFaltantes.drop(columns = 'id', inplace = True)

In [237]:
# Testando de "id" sumiu mesmo!
# Resultado: sim, sumiu!
# df_parcial_dadosObject['id']

In [238]:
# Primeiro vamos fazer uma cópia do DataFrame (df_market_todoNumerico)
# Vamos adicionar o DataFrame resultado depois de inserir todos os dados faltantes do tipo "bool"
# Adicionando o DataFrame (df_parcial_dadosObject) ao DataFrame (df_market_todoNumerico)
# OBS: a coluna "id"até o momento naão esta fazendo parte de nenhum DataFrame!
df_market_todoNumerico_COPIA = df_market_todoNumerico
df_market_todoNumerico_COPIA = df_market_todoNumerico_COPIA.join(df_parcial_dadosObject_SEM_dadosFaltantes)

In [239]:
df_market_todoNumerico_COPIA.shape

(462298, 130)

### >>> Informações! <<<

In [240]:
# Vamos usar o DataFrame (df_market_todoNumerico_COPIA) para realizar as operações de descobrir os valores faltantes!
# Este DataFrame está completo.
# Os valores nulos foram substituidos por: 'souNulo'

In [241]:
from sklearn import tree

In [242]:
clf = tree.DecisionTreeClassifier()

In [243]:
# (lista_dadosObject_COM_dadosFaltantes) : contém todas as colunas do tipo "bool"que tem dados faltantes
lista_dadosObject_COM_dadosFaltantes

['nm_divisao',
 'nm_segmento',
 'fl_optante_simples',
 'de_saude_tributaria',
 'de_saude_rescencia',
 'de_nivel_atividade',
 'de_faixa_faturamento_estimado_grupo']

In [244]:
df_market_dadosObject_COPIA = df_market_dadosObject

In [245]:
from sklearn import preprocessing

In [246]:
le = preprocessing.LabelEncoder()

In [247]:
# Separando as Bases de TREINO e TESTE

for x in lista_dadosObject_COM_dadosFaltantes:
    print('Começando a coluna: ',x)
    
    df_total = df_market_todoNumerico_COPIA.join(df_market_dadosObject[x])

    df_market_TN_treino = df_total[df_total[x] != 'souNulo']
    df_market_TN_teste = df_total[df_total[x] == 'souNulo']
    
    # Definindo o TREINO
    Y_TREINO_coluna = le.fit_transform(df_market_TN_treino[x])
    X_TREINO_coluna = df_market_TN_treino.drop(columns = x)

    # Definindo TESTE
    X_TESTE_coluna = df_market_TN_teste.drop(columns = x)
    
    clf.fit(X_TREINO_coluna, Y_TREINO_coluna)
    Y_PREDITO = clf.predict(X_TESTE_coluna)
    Y_PREDITO_RESULTADO = le.inverse_transform(Y_PREDITO)
    
    # Substituindo 'souNulo' pelos valores calculado'
    df_market_dadosObject_COPIA[x].replace({'souNulo': Y_PREDITO_RESULTADO}, inplace = True)


Começando a coluna:  nm_divisao
Começando a coluna:  nm_segmento
Começando a coluna:  fl_optante_simples
Começando a coluna:  de_saude_tributaria
Começando a coluna:  de_saude_rescencia
Começando a coluna:  de_nivel_atividade
Começando a coluna:  de_faixa_faturamento_estimado_grupo


In [248]:
# Agora (df_market_dadosObject_COPIA) está completa!
# Vamos transformar este DataFrame em um dataFrame numérico com o (get_dummies)

In [249]:
df_market_dadosObject_COPIA.columns

Index(['id', 'de_natureza_juridica', 'sg_uf', 'natureza_juridica_macro',
       'de_ramo', 'idade_emp_cat', 'fl_rm', 'nm_divisao', 'nm_segmento',
       'fl_optante_simples', 'de_saude_tributaria', 'de_saude_rescencia',
       'de_nivel_atividade', 'de_faixa_faturamento_estimado_grupo'],
      dtype='object')

In [250]:
df_market_dadosObject = df_market_dadosObject_COPIA

In [251]:
df_market_dadosObject.drop(columns = 'id', inplace = True)

In [252]:
df_market_dadosObject_transformado = pd.get_dummies(df_market_dadosObject)

In [253]:
df_market_dadosObject_transformado.shape

(462298, 256)

In [254]:
df_market_dadosObject_transformado.columns

Index(['fl_optante_simples', 'de_natureza_juridica_ASSOCIACAO PRIVADA',
       'de_natureza_juridica_AUTARQUIA ESTADUAL OU DO DISTRITO FEDERAL',
       'de_natureza_juridica_AUTARQUIA FEDERAL',
       'de_natureza_juridica_AUTARQUIA MUNICIPAL',
       'de_natureza_juridica_CANDIDATO A CARGO POLITICO ELETIVO',
       'de_natureza_juridica_CLUBE FUNDO DE INVESTIMENTO',
       'de_natureza_juridica_COMUNIDADE INDIGENA',
       'de_natureza_juridica_CONDOMINIO EDILICIO',
       'de_natureza_juridica_CONSORCIO DE EMPREGADORES',
       ...
       'de_faixa_faturamento_estimado_grupo_ATE R$ 81.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 1.500.000,01 A R$ 4.800.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 10.000.000,01 A R$ 30.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 100.000.000,01 A R$ 300.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 30.000.000,01 A R$ 100.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 300.00

In [255]:
df_auxiliar_fl_optante_simples = df_market_dadosObject_transformado['fl_optante_simples']

In [256]:
df_market_dadosObject_transformado.drop(columns= 'fl_optante_simples', inplace = True)

In [257]:
df_market_dadosObject_transformado.columns

Index(['de_natureza_juridica_ASSOCIACAO PRIVADA',
       'de_natureza_juridica_AUTARQUIA ESTADUAL OU DO DISTRITO FEDERAL',
       'de_natureza_juridica_AUTARQUIA FEDERAL',
       'de_natureza_juridica_AUTARQUIA MUNICIPAL',
       'de_natureza_juridica_CANDIDATO A CARGO POLITICO ELETIVO',
       'de_natureza_juridica_CLUBE FUNDO DE INVESTIMENTO',
       'de_natureza_juridica_COMUNIDADE INDIGENA',
       'de_natureza_juridica_CONDOMINIO EDILICIO',
       'de_natureza_juridica_CONSORCIO DE EMPREGADORES',
       'de_natureza_juridica_CONSORCIO DE SOCIEDADES',
       ...
       'de_faixa_faturamento_estimado_grupo_ATE R$ 81.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 1.500.000,01 A R$ 4.800.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 10.000.000,01 A R$ 30.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 100.000.000,01 A R$ 300.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 30.000.000,01 A R$ 100.000.000,00',
       'de_faixa_fatura

In [258]:
df_auxiliar_fl_optante_simples = df_auxiliar_fl_optante_simples.astype(int)

In [259]:
df_auxiliar_fl_optante_simples.unique()

array([1, 0])

In [260]:
df_market_dadosObject_transformado = df_market_dadosObject_transformado.join(df_auxiliar_fl_optante_simples)

In [261]:
df_market_dadosObject_transformado.shape

(462298, 256)

In [262]:
# Agora todos os dados estão completos!
# Vamos juntar todos os DataFrames!

## JUNTANDO TODOS OS DATAFRAMES TRATADOS!

In [263]:
# Nosso DataFrame final vai ser o (df_dadosTratados)

In [264]:
# Primeiro estamos adicionando os valores do ID
df_dadosTratados = pd.DataFrame(armazenando_ID)

In [265]:
df_dadosTratados.shape

(462298, 1)

In [266]:
# Fazendo JOIN (df_dadosTratados) com os dados do tipo: "bool"
df_dadosTratados = df_dadosTratados.join(df_market_dadosBool_transformado)

In [267]:
# Fazendo JOIN (df_dadosTratados) com os dados do tipo: "int64" e "float64"
df_dadosTratados = df_dadosTratados.join(df_market_dadosNumericos_Transformado)

In [268]:
# Fazendo JOIN (df_dadosTratados) com os dados do tipo: object (parcial_semFaltantes)
df_dadosTratados = df_dadosTratados.join(df_market_dadosObject_transformado)

In [269]:
df_dadosTratados.shape

(462298, 266)

In [270]:
df_dadosTratados.columns

Index(['id', 'fl_matriz', 'fl_sa', 'fl_mei', 'fl_ltda', 'fl_email',
       'fl_telefone', 'idade_empresa_anos', 'nu_meses_rescencia',
       'vl_faturamento_estimado_aux',
       ...
       'de_faixa_faturamento_estimado_grupo_DE R$ 1.500.000,01 A R$ 4.800.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 10.000.000,01 A R$ 30.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 100.000.000,01 A R$ 300.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 30.000.000,01 A R$ 100.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 300.000.000,01 A R$ 500.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 360.000,01 A R$ 1.500.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 4.800.000,01 A R$ 10.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 500.000.000,01 A 1 BILHAO DE REAIS',
       'de_faixa_faturamento_estimado_grupo_DE R$ 81.000,01 A R$ 360.000,00',
       'fl_optante_simples'],
      dtype='object', length=

## Salvando o DataFrame numérico num arquivo para facilitar a vida!

In [271]:
df_dadosTratados.to_csv('dataframe_populacao_tratada.csv')